[参考](https://www.kaggle.com/abhishek/beating-the-benchmark)。

chenglong-chen 获得了第 1 名：
http://blog.kaggle.com/2015/07/27/crowdflower-winners-interview-1st-place-chenglong-chen/

https://github.com/ChenglongChen/Kaggle_CrowdFlower


https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html#sklearn.metrics.cohen_kappa_score

In [1]:
import pandas as pd

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [2]:
train.head()

,id,query,product_title,product_description,median_relevance,relevance_variance
0,1,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,2,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,4,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471
3,5,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,7,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471


In [3]:
test.head()

,id,query,product_title,product_description
0,3,electric griddle,Star-Max 48 in Electric Griddle,NaN
1,6,phillips coffee maker,Philips SENSEO HD7810 WHITE Single Serve Pod C...,NaN
2,9,san francisco 49ers,2013 San Francisco 49ers Clock,A 2013 San Francisco 49ers clock is the ultima...
3,11,aveeno shampoo,AVEENO 10.5FLOZ NRSH SHINE SH,"Water, Ammonium Lauryl Sulfate, Dimethicone, S..."
4,12,flea and tick control for dogs,Merial Frontline Plus Flea and Tick Control fo...,NaN


In [2]:
print(train.shape)
print(test.shape)

(10158, 6)
(22513, 4)


In [3]:
idx = test.id
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [4]:
train.head()

,query,product_title,product_description,median_relevance,relevance_variance
0,bridal shower decorations,Accent Pillow with Heart Design - Red/Black,Red satin accent pillow embroidered with a hea...,1,0.000
1,led christmas lights,Set of 10 Battery Operated Multi LED Train Chr...,Set of 10 Battery Operated Train Christmas Lig...,4,0.000
2,projector,ViewSonic Pro8200 DLP Multimedia Projector,NaN,4,0.471
3,wine rack,Concept Housewares WR-44526 Solid-Wood Ceiling...,"Like a silent and sturdy tree, the Southern En...",4,0.000
4,light bulb,Wintergreen Lighting Christmas LED Light Bulb ...,"WTGR1011\nFeatures\nNickel base, 60,000 averag...",2,0.471


In [5]:
# 只要中位数，不要方差
y = train.median_relevance.values
train = train.drop(['median_relevance', 'relevance_variance'], axis=1)

## 把 query 和 product_title 拼接在一起

+ product_description 没有用。

In [6]:
%%time
# do some lambda magic on text columns
traindata = list(
    train.apply(
        lambda x: '{} {}'.format(x['query'], x['product_title']), axis=1))
testdata = list(
    test.apply(
        lambda x: '{} {}'.format(x['query'], x['product_title']), axis=1))

CPU times: user 668 ms, sys: 5.06 ms, total: 673 ms
Wall time: 673 ms


In [7]:
print(traindata[0])
print(traindata[1])

bridal shower decorations Accent Pillow with Heart Design - Red/Black
led christmas lights Set of 10 Battery Operated Multi LED Train Christmas Lights - Clear Wire


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# the infamous tfidf vectorizer (Do you remember this one?)
tfv = TfidfVectorizer(
    min_df=3,
    max_features=None,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 5),
    use_idf=1,
    smooth_idf=1,
    sublinear_tf=1,
    stop_words='english')

In [9]:
%%time
# Fit TFIDF
tfv.fit(traindata)

CPU times: user 1.16 s, sys: 47.5 ms, total: 1.21 s
Wall time: 1.21 s


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 5), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words='english', strip_accents='unicode', sublinear_tf=1,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=1,
        vocabulary=None)

In [10]:
X = tfv.transform(traindata)
X_test = tfv.transform(testdata)

In [11]:
print(X.toarray().shape)
print(X_test.toarray().shape)

(10158, 19255)
(22513, 19255)


In [12]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

svd = TruncatedSVD()
scl = StandardScaler()
svm_model = SVC()
clf = Pipeline([('svd', svd), ('scl', scl), ('svm', svm_model)])

https://www.jianshu.com/p/37e689bab29b

https://blog.csdn.net/weixin_34128501/article/details/87082786

http://sofasofa.io/forum_main_post.php?postid=1000321

计算二次加权 kappa
    axquadratic_weighted_kappa计算二次加权kappa
    价值，衡量两个评估者之间的评估者之间的协议
    提供离散数字评级。潜在值范围为-1
    （代表完全不同意）到1（代表完整
    协议）。如果所有协议都是由于，则预计kappa值为0
    机会。
    quadratic_weighted_kappa（rater_a，rater_b），其中rater_a和rater_b
    每个对应一个整数评级列表。这些清单必须有
    相同的长度。
    评级应该是整数，并且假设它们包含
    完整的可能评级范围。
    quadratic_weighted_kappa（X，min_rating，max_rating），其中min_rating
    是最小可能的评级，max_rating是最大可能的评级
    评分

In [13]:
import numpy as np


# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert (len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)] for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating = None
    max_rating = None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert (len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b, min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (
                hist_rater_a[i] * hist_rater_b[j] / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [14]:
from sklearn.metrics import make_scorer

# Kappa Scorer
kappa_scorer = make_scorer(quadratic_weighted_kappa, greater_is_better=True)

In [15]:
# Create a parameter grid to search for best parameters for everything in the pipeline
param_grid = {'svd__n_components': [200, 400], 'svm__C': [10, 12]}

In [16]:
%%time

from sklearn.model_selection import GridSearchCV

model = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring=kappa_scorer,
    verbose=10,
    n_jobs=-1,
    iid=True,
    refit=True,
    cv=2)

model.fit(X, y)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:   33.2s remaining:  1.7min
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:   33.2s remaining:   55.4s
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed:   33.4s remaining:   33.4s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:   51.4s remaining:   30.8s
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:   53.1s remaining:   17.7s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   54.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   54.5s finished


CPU times: user 58.1 s, sys: 2 s, total: 1min
Wall time: 1min 44s


In [17]:
print("Best score: {:.3f}".format(model.best_score_))

Best score: 0.559


In [18]:
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()

Best parameters set:


In [19]:
for param_name in sorted(param_grid.keys()):
    print("\t{}: {}".format(param_name, best_parameters[param_name]))

	svd__n_components: 400
	svm__C: 12


In [20]:
# Get best model
best_model = model.best_estimator_

In [21]:
# Fit model with best parameters optimized for quadratic_weighted_kappa
best_model.fit(X, y)
preds = best_model.predict(X_test)

In [22]:
# Create your first submission file
submission = pd.DataFrame({"id": idx, "prediction": preds})
submission

,id,prediction
0,3,4
1,6,4
2,9,3
3,11,4
4,12,4
5,14,4
6,15,4
7,16,4
8,18,4
9,19,3


In [23]:
submission.to_csv("../output/beating_the_benchmark_yet_again.csv", index=False)